# Sobre este archivo

La primera enterga pretende ser principalmente exploratoria,
quizás nada del código que aparece aquí llegue a la versión final.

La idea es investigar como se podrían hacer las funciones principales.
Conviene entonces intentar separa el proyecto.
Una esquema preliminar es el siguiente.

## ¿Qué hay que hacer?

Usar algoritmos genéticos para aproximar una imágen usando figuras.
Inicialmente estas figuras son triángulos, quizás esto cambie.

Se pueden ver los siguientes pasos necesarios:
1. Leer la imagén del disco y cargarla en python
2. 